
# Exercise 6: Water vapor retrieval from airborne observations

Manfred Brath, Oliver Lemke

In [ ]:
%matplotlib widget

import os
import matplotlib.pyplot as plt
import numpy as np
from pyarts import xml
from nonlin_oem import Forward_model, set_correlation_length, create_apriori_covariance_matrix, watervapor_retrieval

os.makedirs("plots", exist_ok=True)

In this exercise we want to retrieve water vapor profiles from (simulated) 
airborne microwave radiometer observation using the optimal estimation method.
The radiometer is the HAMP radiometer on board the HALO aircraft.
The radiometer measures the brightness temperature at several set of channels.
In this exercise we will use the sets of channels around 22GHz, 50GHz, 90GHz, 118GHz and 183GHz.
The radiometer is mounted in the belly pod of the HALO aircraft and measures the 
brightness temperature at nadir.
We will use a simplified version of the HAMP radiometer but with the correct
channels and NeDT.

![HALO aircraft](Halo.jpg)
*source https://halo-research.de/ressources/image-galery/*

The NeDT (Noise equivalent delta temperature) are the following:
Channel set           | NeDT (K)
----------------------|--------:
22 GHz channels       |    0.10
50 GHz channels       |    0.20
90 GHz channel        |    0.25
118 GHz channels      |    0.60
183 GHz channels      |    0.60

The measurement data consists of a short ($\approx$ 200km) flight segment of the HALO aircraft at clear sky conditions over the tropical pacific.
The flight segment is at 15km altitude. The measurement data consists of brightness temperature observations for the three sets of channels.
Each data set consists of a file that contains the measurement data (*y_obs_xxxGHz.xml*), a file with the sensor characteristics (*SensorCharacteristics_xxxGHz.xml*) and a file with the latitude of the measurements (*lat.xml*).
Furthermore there also exists dropsonde measurement data (*dropsonde_Wv.xml*) from one dropsonde that was released during the flight segment.  The dropsonde data contains the temperature, altitude and H2O vmr profiles as function of pressure. 
The measurement data is stored in the directory *observation*.
The surface temperature during that flight segment was 300K. The surface reflectivity is 0.4 for all frequencies for that flight segment.

### Part I - Preparations

#### 1) 
Read in the measurement data and sensor characteristics. Plot the brightness temperature observations for the five sets of channels as function of latitude. Furthermore plot the dropsonde temperature profile.
Depending on the channel set, the actual channel consists of a single center frequency or of at least two sidebands channels around the center frequency.

In [ ]:
# Load the observation data
sensor_characteristics_22GHz = xml.load("observation/SensorCharacteristics_22GHz.xml") # sensor characteristics

# The sensor characteristics is a matrix with the following columns:
# 0: Frequency in GHz
# 1: sideband offset1 to center frequency in GHz
# 2: sideband offset2 to center frequency in GHz
# 3: bandwidth in GHz
# 4: relative mandatory frequency grid spacing for the passbands
#
# The rows of the matrix are the different channels


y_obs_22GHz = xml.load("observation/y_obs_22GHz.xml.xml")[:] # [:] converts the data to a numpy array
#...


#...and the dropsonde data
dropsonde = xml.load("observation/dropsonde_WV.xml")

# dropsonde.grids[0] gives you the name of the variables in the dropsonde file
# Use dropsonde.get("VARIABLENAME", keep_dims=False) to get the data of the variable VARIABLENAME
# dropsonde.grids[1][:] gives the pressure grid

# Plot the observation data

#### 2)
Decide and explain which set of channels you want to use for the water vapor retrieval.
If you want you can use the dropsonde data and the function forward model to simulate the brightness temperatures  and jacobians for the dropsonde temperature profile, but **you don't have to**.

In [ ]:
# y_obs, jacobians = Forward_model([], dropsonde_data,..., sensor_description=sensor_characteristics)


#### 3)
Prepare the covariance matrices for the water vapor retrieval.
Use the function *create_apriori_covariance_matrix* to create the a priori covariance matrix. The function assumes that an exponentially decaying correlation function is used. 

You can use the function *set_correlation_length(z, len_sfc, len_toa=None)* to set the correlation length for the a priori covariance matrix. You can use a constant or a linearly increasing correlation length with height.
Make an educated guess for the correlation length. Remember that the flight segment is over the tropical pacific.

* Set the a priori covaraince matrix for the water vapor retrieval.
* Set the measurement error covariance matrix using the NeDT values. Assume a diagonal matrix.
* Plot the covariance matrices in suitable way.

One remark, as vmr of water vapor spans a range over several orders of magnitude, the retrieval must be done in log-space. This means that the state vector consists of  $\textrm{log}_{10}$ (vmr) values. In this case the covariance matrix must also defined in log-space.

In [ ]:
# correlation_length = set_correlation_length(z, len_sfc, len_toa)
# S_a = create_apriori_covariance_matrix(x, z, delta_x, correlation_length)

# S_y = ...

### Part II - Retrieval


#### 4)
Use the function *water_vapor_retrieval* to retrieve the water vapor profile from the first brightness temperature observation of the flight segment. The function has the following signature:
```python   
   wv_ret, DeltaT, y_fit = temperature_retrieval(y_obs, f_grid, sensor_pos, sensor_los, background_atmosphere, surface_temperature, surface_reflectivity, S_y, S_a, senssor_description=[], Diagnostics=False) 
```     
*sensor_pos*, *sensor_los*, *background_atmosphere*, *surface_temperature*, *surface_reflectivity* describe the background state of the atmosphere and the sensor position and line of sight. 
*background_atmosphere* has a double function. It includes the background atmospheric state (e. g. water vapor profile) for the non retrieved atmospheric variables and the a priori temperature profile. 
If *sensor_description* is set to *[]* then the function uses f_grid. If *sensor_description* is set then the sensor description is used.

The function returns the retrieved water vapor profile, the total error of the retrieved water vapor profile and the fitted brightness temperature measurement.

Use the prepared covariance matrices for the retrieval and the dropsonde data as background state and a priori water vapor profile.

Check the results:
* Compare the retrieved water vapor profile with the dropsonde water vapor profile, which you used as a priori.
* Check the fit of the brightness temperature measurement.
* Compare the total error of the retrieved water vapor profile with the square root of the components of the diagonal of the a priori covariance matrix.
* Where does the retrieval gain most information? Use the diagnostics output of the function *water_vapor_retrieval*. To do that, set the keyword *Diagnostics=True* in the function *water_vapor_retrieval* and add *A* and *G* to the output of the function.

In [ ]:
# wv_ret, DeltaT, y_fit = watervapor_retrieval(
                                            # y_obs,
                                            # [],
                                            # sensor_pos,
                                            # sensor_los,
                                            # dropsonde,
                                            # surface_temperature,
                                            # surface_reflectivity,
                                            # S_y,
                                            # S_a,
                                            # sensor_description=sensor_characteristics)

#### 5)
Repeat the retrieval for the rest of the flight segment. 

* Plot the retrieved water vapor profiles and the difference to the a priori as function of altitude and latitude. 
* Plot the total error of the retrieved water vapor profiles as function of altitude and latitude. 
* Plot the difference between the fitted and measured brightness temperature (residuals) as function of latitude.
* Explain your results.



